In [1]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup

In [2]:
page = wptools.page('Palestinian_insurgency_in_South_Lebanon')
page.get_parse()

en.wikipedia.org (parse) Palestinian_insurgency_in_South_Lebanon
en.wikipedia.org (imageinfo) File:FatehMilitia.jpg
Palestinian insurgency in South Lebanon (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:FatehMili...
  infobox: <dict(14)> conflict, partof, image, image_size, caption...
  pageid: 35391139
  parsetree: <str(21674)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: Palestinian insurgency in South Lebanon
  wikibase: Q7127419
  wikidata_url: https://www.wikidata.org/wiki/Q7127419
  wikitext: <str(16219)> {{Short description|1968–1982 conflict in...
}


In [3]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Palestinian_insurgency_in_South_Lebanon",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Palestinian insurgency in South Lebanon')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1958 Lebanon Crisis
1968 Israeli raid on Lebanon
1972 Israeli air raid in Syria and Lebanon
1973 Athens Hellinikon International Airport attack
1973 Israeli raid on Lebanon
1973 New York City bomb plot
1974 Nahariya attack
1978 South Lebanon conflict
1979 Nahariya attack
1982 Lebanon War
1982 Lebanon war
Abu Nidal organization
Amal Movement
Antoine Lahad
Arab Cold War
Arab–Israeli conflict
Ariel Sharon
Athens International Airport
Attack on the Saudi Embassy in Khartoum
Avivim school bus bombing
Avivim school bus massacre
Bachir Gemayel
Beirut
Beirut International Airport
Black September in Jordan
Bombing of Lebanon (June 1981)
Cairo Agreement (1969)
Cedar Revolution
Coastal Road massacre
Democratic Front for the Liberation of Palestine
Demographics of Lebanon
Displaced persons
El Al Flight 253 attack
El Al Flight 426 hijacking
El Al Flight 432 attack
Ephraim Katzir
Fatah
Good Fence
Hafez al-Assad
Haifa, Israel
Hezbollah
Hussein of Jordan
ISBN (identifier)
Iran
Israel
Israeli Ba

In [4]:
def get_arabic_name(english_phrase):
    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries
    
    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s




In [5]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [6]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [7]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1
                    
                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    if isinstance(name, list):
                        if len(name) > 1:
                            for n in name:
                                print('name before: ', n)
                                name = [n]
                                break
#                             name = list(name[0])
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'
                    
                    df_political_party_en = df_political_party_en.append({
                        'Name': ','.join(name if isinstance(name, list) else name),
                        'Ideology': ','.join(ideology) if isinstance(ideology, list) else ideology,
                        'Political Position': ','.join(position) if isinstance(position, list) else position,
                        'Founder(s)': founders,
                    }, ignore_index=True)
                    
                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)
                    
                    
                    
                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']
                    
                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'
                    
                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')
                    
                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'
                    
                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')
                    
                    
                    df_political_party_ar = df_political_party_ar.append({
                        'الاسم': ','.join(name_ar),
                        'الإيديولوجيا': ','.join(ideology_ar),
                        'الموقف السياسي': ','.join(position_ar),
                        'المؤسس (المؤسسون)': ','.join(founders_ar),
                    }, ignore_index=True)
                    
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1958 Lebanon Crisis
en.wikipedia.org (imageinfo) File:Foxhole - Lebanon - Beirut - Ju...
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1972 Israeli air raid in Syria and Lebanon
en.wikipedia.org (parse) 1973 Athens Hellinikon International Air...
en.wikipedia.org (parse) 1973 Israeli raid on Lebanon
en.wikipedia.org (parse) 1973 New York City bomb plot
en.wikipedia.org (parse) 1974 Nahariya attack
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1979 Nahariya attack
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 Lebanon war
en.wikipedia.org (parse) Abu Nidal organization


-----------------
name:  ['Fatah – The Revolutionary Council']
founders:  
ideology:  ['Palestinian nationalism', 'Anti-Zionism', 'Secularism']
position:  -
START
Cannot find the link: http://en.wikipedia.org/wiki/Fatah – The Revolutionary Council
name:  Fatah – The Revolutionary Council
name ar:  فتح - المجلس الثوري (مترجمه)
name ar link:  -1
END
START
ideology:  Palestinian nationalism
ideology ar:  قومية فلسطينية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
ideology:  Anti-Zionism
ideology ar:  معاداة الصهيونية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B5%D9%87%D9%8A%D9%88%D9%86%D9%8A%D8%A9
ideology:  Secularism
ideology ar:  علمانية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85%D8%A7%D9%86%D9%8A%D8%A9
END
START
END


en.wikipedia.org (parse) Amal Movement


[[Nabih Berri]]
[[Musa al-Sadr]]<br /> [[Hussein el-Husseini]]
-----------------
name:  ['Amal Movement']
founders:  Nabih Berri,Hussein el-Husseini,Musa al-Sadr,
ideology:  ['Populism', 'Conservatism']
position:  ['Centre-right']
START
name:  Amal Movement
name ar:  حركة أمل 
name ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B1%D9%83%D8%A9_%D8%A3%D9%85%D9%84
END
START
ideology:  Populism
ideology ar:  شعبوية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B4%D8%B9%D8%A8%D9%88%D9%8A%D8%A9
ideology:  Conservatism
ideology ar:  سياسة محافظة 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9_%D9%85%D8%AD%D8%A7%D9%81%D8%B8%D8%A9
END
START
position:  Centre-right
position ar:  وسط اليمين 
position ar link:  https://ar.wikipedia.org/wiki/%D9%88%D8%B3%D8%B7_%D8%A7%D9%84%D9%8A%D9%85%D9%8A%D9%86
END
START
founder:  Nabih Berri
founder ar:  نبيه بري 
founder ar link:  https://ar.wikipedia.org/wiki/%D9%86%D8%A8%D9%8A%D9%87_%D8%A8%D8%B1%D9%8A
Cannot translate phras

en.wikipedia.org (parse) Antoine Lahad
en.wikipedia.org (imageinfo) File:Antoine Lahad.jpg
en.wikipedia.org (parse) Arab Cold War
en.wikipedia.org (parse) Arab–Israeli conflict
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon Headshot.jpg
en.wikipedia.org (parse) Athens International Airport
en.wikipedia.org (imageinfo) File:Athens airport logo.svg
en.wikipedia.org (parse) Attack on the Saudi Embassy in Khartoum
en.wikipedia.org (parse) Avivim school bus bombing
en.wikipedia.org (imageinfo) File:Avivim school bus bombing I.jpg
en.wikipedia.org (parse) Avivim school bus massacre
en.wikipedia.org (imageinfo) File:Avivim school bus bombing I.jpg
en.wikipedia.org (parse) Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bachir With His Father Pierre (...
en.wikipedia.org (parse) Beirut
en.wikipedia.org (parse) Beirut International Airport
en.wikipedia.org (imageinfo) File:BeirutAirportLogo.png
en.wikipedia.org (parse) Black September in Jordan
en.wikipedi

...

[[Mahmoud Abbas]]
-----------------
name:  ['Palestine Liberation Organization']
founders:  Mahmoud Abbas,
ideology:  ['Palestinian nationalism']
position:  -
START
name:  Palestine Liberation Organization
name ar:  منظمة التحرير الفلسطينية 
name ar link:  https://ar.wikipedia.org/wiki/%D9%85%D9%86%D8%B8%D9%85%D8%A9_%D8%A7%D9%84%D8%AA%D8%AD%D8%B1%D9%8A%D8%B1_%D8%A7%D9%84%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
END
START
ideology:  Palestinian nationalism
ideology ar:  قومية فلسطينية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
END
START
Cannot translate phrase `Mahmoud Abbas` to arabic
END


en.wikipedia.org (parse) Palestinian Fedayeen insurgency
en.wikipedia.org (imageinfo) File:Fedayeen 1956.jpg
en.wikipedia.org (parse) Palestinian fedayeen
en.wikipedia.org (parse) Pan Am Flight 110
en.wikipedia.org (parse) Philip Habib
en.wikipedia.org (imageinfo) File:Philip Habib.jpg
en.wikipedia.org (parse) Popular Front for the Liberation of Pale...


[[Ahmed Jibril]]
-----------------
name:  ['Popular Front for the Liberation of Palestine - General Command']
founders:  Ahmed Jibril,
ideology:  ['Palestinian nationalism', 'Arab nationalism']
position:  -
START
name:  Popular Front for the Liberation of Palestine - General Command
name ar:  الجبهة الشعبية لتحرير فلسطين – القيادة العامة 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AC%D8%A8%D9%87%D8%A9_%D8%A7%D9%84%D8%B4%D8%B9%D8%A8%D9%8A%D8%A9_%D9%84%D8%AA%D8%AD%D8%B1%D9%8A%D8%B1_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86_%E2%80%93_%D8%A7%D9%84%D9%82%D9%8A%D8%A7%D8%AF%D8%A9_%D8%A7%D9%84%D8%B9%D8%A7%D9%85%D8%A9
END
START
ideology:  Palestinian nationalism
ideology ar:  قومية فلسطينية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
ideology:  Arab nationalism
ideology ar:  قومية عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
END
STAR

en.wikipedia.org (parse) Rafael Eitan
en.wikipedia.org (imageinfo) File:Rafael Eitan, Chief of General ...
en.wikipedia.org (parse) Rejectionist Front
en.wikipedia.org (parse) Saad Haddad
en.wikipedia.org (parse) Sabena Flight 571
en.wikipedia.org (imageinfo) File:Sabena Flight 571. I.jpg
en.wikipedia.org (parse) Sabena Flight 571 hijacking
en.wikipedia.org (imageinfo) File:Sabena Flight 571. I.jpg
en.wikipedia.org (parse) Samir Kuntar
en.wikipedia.org (imageinfo) File:Samir kuntar in shiraz 3-2.JPG
en.wikipedia.org (parse) Savoy Hotel attack
en.wikipedia.org (imageinfo) File:Savoy Hotel attack. XXII.jpg
en.wikipedia.org (parse) Sayeret Matkal
en.wikipedia.org (imageinfo) File:Sayeret Matkal insignia.svg
en.wikipedia.org (parse) Shlomo Argov
en.wikipedia.org (parse) Siege of Beirut
en.wikipedia.org (parse) South Lebanese Army
en.wikipedia.org (imageinfo) File:Flag of the South Lebanon Army.png
en.wikipedia.org (parse) South Lebanon Army
en.wikipedia.org (imageinfo) File:Flag of the Sou

skipping Wikipedia:Protection policy
skipping Wikipedia:Verifiability
skipping Template:Campaignbox Palestinian insurgency in South Lebanon
skipping Template:Country data Israel
skipping Template:Country data PLO
skipping Template:Country data Syria
skipping Template:Country data USSR
skipping Template talk:Campaignbox Palestinian insurgency in South Lebanon
skipping Category:Use dmy dates from November 2020
skipping Category:Wikipedia articles needing factual verification from November 2011


In [8]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Fatah – The Revolutionary Council,"Palestinian nationalism,Anti-Zionism,Secularism",-,
1,Amal Movement,"Populism,Conservatism",Centre-right,"Nabih Berri,Hussein el-Husseini,Musa al-Sadr,"
2,Democratic Front for the Liberation of Palestine,"Communism,Palestinian nationalism,ubl,Marxism–...",Far-left,
3,Fatah,"Palestinian nationalism,Social democracy,Secul...","Centre-left,left-wing","Mahmoud Aloul,"
4,Hezbollah,"Anti-imperialism,Shia Jihad,Anti-West,Pan-Isla...",-,"Sayyed Hassan Nasrallah,"
5,Lebanese Kataeb Party,"Social conservatism,Anti-communism,Lebanese na...","Right-wing,far-right,Centre-right","Samy Gemayel,Pierre Gemayel,"
6,Lebanese Forces,"Christian nationalism,National conservatism,Le...",Right-wing,"Samir Geagea,Bachir Gemayel,"
7,ar,"Anti-pan-Arabism,Anti-communism,Lebanese natio...",-,
8,Palestine Liberation Organization,Palestinian nationalism,-,"Mahmoud Abbas,"
9,Palestinian Liberation Front,Arab nationalism,-,


In [9]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,فتح - المجلس الثوري (مترجمه),"قومية فلسطينية ,معاداة الصهيونية ,علمانية",-,
1,حركة أمل,"شعبوية ,سياسة محافظة",وسط اليمين,نبيه بري
2,الجبهة الديمقراطية لتحرير فلسطين,"شيوعية ,قومية فلسطينية ,ماركسية لينينية ,معادا...",أقصى اليسار,
3,حركة فتح,"قومية فلسطينية ,ديمقراطية اجتماعية ,علمانية","اليسار المعتدل ,يسارية",
4,حزب الله,"معاداة الاستعمارية ,جهاد الشيعة (مترجمه),وحدة ...",-,حسن نصر الله
5,حزب الكتائب اللبنانية (مترجمه),"محافظة اجتماعية ,معاداة الشيوعية ,قومية لبناني...","يمينية ,يمين متطرف ,وسط اليمين","سامي الجميل ,بيار الجميل"
6,القوات اللبنانية,"قومية مسيحية ,محافظة وطنية ,قومية لبنانية ,محا...",يمينية,"سمير جعجع ,بشير الجميل"
7,مع (مترجمه),"معاداة القومية العربية (مترجمه),معاداة الشيوعي...",-,
8,منظمة التحرير الفلسطينية,قومية فلسطينية,-,
9,جبهة التحرير الفلسطينية,قومية عربية,-,


In [10]:
datasets_dir = 'datasets_updated/1982-1984/Palestinian_insurgency_in_South_Lebanon/'

In [11]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [12]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [13]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'
                
                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'
                    
                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                    
                    df_politician_en = df_politician_en.append({
                        'Name': name_inserted,
                        'Political Party': ','.join(party) if isinstance(party, list) else party,
                        'Nationality': ','.join(nationality) if isinstance(nationality, list) else nationality,
                        'Religion': ','.join(religion) if isinstance(religion, list) else religion,
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)
                    
                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)
                    
                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'
                    
                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'
                        
                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'
                    
                    df_politician_ar = df_politician_ar.append({
                        'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                        'الحزب السياسي': ','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar,
                        'الجنسيه': ','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar,
                        'الدين': ','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar,
                    }, ignore_index=True)
                    
                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)

        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1958 Lebanon Crisis
en.wikipedia.org (imageinfo) File:Foxhole - Lebanon - Beirut - Ju...
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1972 Israeli air raid in Syria and Lebanon
en.wikipedia.org (parse) 1973 Athens Hellinikon International Air...
en.wikipedia.org (parse) 1973 Israeli raid on Lebanon
en.wikipedia.org (parse) 1973 New York City bomb plot
en.wikipedia.org (parse) 1974 Nahariya attack
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1979 Nahariya attack
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 Lebanon war
en.wikipedia.org (parse) Abu Nidal organization
en.wikipedia.org (parse) Amal Movement
en.wikipedia.org (parse) Antoine Lahad
en.wikipedia.org (imageinfo) File:Antoine Lahad.jpg


-----------------
name:  ['Antoine Lahad']
native_name:  -
nickname:  -
religion:  -
party:  -
nationality:  -
name ar:  أنطوان لحد 
name ar:  أنطوان لحد 
religion ar:  -
party ar:  -
nationality ar:  -


en.wikipedia.org (parse) Arab Cold War
en.wikipedia.org (parse) Arab–Israeli conflict
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon Headshot.jpg


-----------------
name:  -
native_name:  -
nickname:  -
religion:  -
party:  ['plainlist']
nationality:  -
name ar:  أرئيل شارون 
START
Cannot find the link: http://en.wikipedia.org/wiki/plainlist
political party:  plainlist
political party ar:  قائمة عادية (مترجمه)
political party ar link:  -1
END
name ar:  أرئيل شارون 
religion ar:  ['قائمة عادية (مترجمه)']
party ar:  ['plainlist']
nationality ar:  -


en.wikipedia.org (parse) Athens International Airport
en.wikipedia.org (imageinfo) File:Athens airport logo.svg
en.wikipedia.org (parse) Attack on the Saudi Embassy in Khartoum
en.wikipedia.org (parse) Avivim school bus bombing
en.wikipedia.org (imageinfo) File:Avivim school bus bombing I.jpg
en.wikipedia.org (parse) Avivim school bus massacre
en.wikipedia.org (imageinfo) File:Avivim school bus bombing I.jpg
en.wikipedia.org (parse) Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bachir With His Father Pierre (...


...

-----------------
name:  ['Yasser Arafat']
native_name:  ['ياسر عرفات']
nickname:  ['Abu Ammar']
religion:  -
party:  ['Fatah']
nationality:  ['Palestinian']
Cannot translate phrase `Yasser Arafat` to arabic
name ar:  -1
START
political party:  Fatah
political party ar:  حركة فتح 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B1%D9%83%D8%A9_%D9%81%D8%AA%D8%AD
END
START
nationality:  Palestinian
nationality ar:  فلسطينيون 
nationality ar link:  https://ar.wikipedia.org/wiki/%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D9%88%D9%86
END
name ar:  -1
religion ar:  ['حركة فتح ']
party ar:  ['Fatah']
nationality ar:  ['فلسطينيون ']


en.wikipedia.org (parse) Yassir Arafat
en.wikipedia.org (imageinfo) File:Flickr - Government Press Offic...


-----------------
name:  ['Yasser Arafat']
native_name:  ['ياسر عرفات']
nickname:  ['Abu Ammar']
religion:  -
party:  ['Fatah']
nationality:  ['Palestinian']
Cannot translate phrase `Yassir Arafat` to arabic
name ar:  -1
START
political party:  Fatah
political party ar:  حركة فتح 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B1%D9%83%D8%A9_%D9%81%D8%AA%D8%AD
END
START
nationality:  Palestinian
nationality ar:  فلسطينيون 
nationality ar link:  https://ar.wikipedia.org/wiki/%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D9%88%D9%86
END
name ar:  -1
religion ar:  ['حركة فتح ']
party ar:  ['Fatah']
nationality ar:  ['فلسطينيون ']


en.wikipedia.org (parse) Yitzhak Navon
en.wikipedia.org (imageinfo) File:Portrait of MP Yitzhak Navon.jpg


-----------------
name:  ['Yitzhak Navon']
native_name:  -
nickname:  -
religion:  -
party:  ['Alignment']
nationality:  ['Israeli']
name ar:  إسحاق نافون 
START
political party:  Alignment
political party ar:  تراصف 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AA%D8%B1%D8%A7%D8%B5%D9%81
END
START
nationality:  Israeli
nationality ar:  إسرائيلي (مترجمه)
nationality ar link:  -1
END
name ar:  إسحاق نافون 
religion ar:  ['تراصف ']
party ar:  ['Alignment']
nationality ar:  ['إسرائيلي (مترجمه)']
skipping Wikipedia:Protection policy
skipping Wikipedia:Verifiability
skipping Template:Campaignbox Palestinian insurgency in South Lebanon
skipping Template:Country data Israel
skipping Template:Country data PLO
skipping Template:Country data Syria
skipping Template:Country data USSR
skipping Template talk:Campaignbox Palestinian insurgency in South Lebanon
skipping Category:Use dmy dates from November 2020
skipping Category:Wikipedia articles needing factual verification from Nove

In [14]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,"(Antoine Lahad,)",-,-,-
1,"(-,)",plainlist,-,-
2,"(بشير الجميّل,Bachir Gemayel,ar,lang,)",Kataeb Party,-,-
3,"(Ephraim Katzir,)",Israeli Labor Party,Israeli,-
4,"(Hafez al-Assad,)",,-,-
5,"(Hussein,)",-,-,Sunni Islam
6,"(-,)",-,-,-
7,"(Philip Habib,)",-,American,-
8,"(-,)",-,-,-
9,"(Saad Haddad,)",-,-,-


In [15]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,-,-,أنطوان لحد
1,-,-,قائمة عادية (مترجمه),أرئيل شارون
2,-,-,حزب الكتائب اللبنانية,بشير الجميل
3,-,إسرائيلي (مترجمه),حزب العمل الإسرائيلي,إفرايم كاتسير
4,-,-,,-1
5,أهل السنة والجماعة,-,-,الحسين بن طلال
6,-,-,-,مناحم بيجن
7,-,أمريكي (مترجمه),-,-1
8,-,-,-,رفائيل إيتان (11 يناير 1929)
9,-,-,-,-1


In [16]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [17]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [18]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)
                    
                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'
    
                    df_locations_en = df_locations_en.append({
                        'Name': ','.join(name) if isinstance(name, list) and len(name) > 1 else name,
                        'Subdivision type': ','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type,
                        'Subdivision name': ','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name                    
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)
                    
                    
                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass
                    
                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'
        

                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = df_locations_ar.append({
                            'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                            'نوع التقسيم': ','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar,
                            'اسم التقسيم': ','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar,
                        }, ignore_index=True)
                    
                    else:
                        print('skipping {} because name is not found'.format(p))
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1958 Lebanon Crisis
en.wikipedia.org (imageinfo) File:Foxhole - Lebanon - Beirut - Ju...
en.wikipedia.org (parse) 1968 Israeli raid on Lebanon
en.wikipedia.org (parse) 1972 Israeli air raid in Syria and Lebanon
en.wikipedia.org (parse) 1973 Athens Hellinikon International Air...
en.wikipedia.org (parse) 1973 Israeli raid on Lebanon
en.wikipedia.org (parse) 1973 New York City bomb plot
en.wikipedia.org (parse) 1974 Nahariya attack
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1979 Nahariya attack
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 Lebanon war
en.wikipedia.org (parse) Abu Nidal organization
en.wikipedia.org (parse) Amal Movement
en.wikipedia.org (parse) Antoine Lahad
en.wikipedia.org (imageinfo) File:Antoine Lahad.jpg
en.wikipedia.org (parse) Arab Cold War
en.wikipedia.org (parse) Arab–Israeli conflict
en.wikipedia.org (parse) 

-----------------
name:  ['Beirut']
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon']
name ar:  بيروت 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) Beirut International Airport
en.wikipedia.org (imageinfo) File:BeirutAirportLogo.png
en.wikipedia.org (parse) Black September in Jordan
en.wikipedia.org (imageinfo) File:Smoke rises above Amman during ...
en.wikipedia.org (parse) Bombing of Lebanon (June 1981)
en.wikipedia.org (parse) Cairo Agreement (1969)
en.wikipedia.org (parse) Cedar Revolution
en.wikipedia.org (parse) Coastal Road massacre
en.wikipedia.org (imageinfo) File:הפיגוע בכביש החוף 1978.jpg
en.wikipedia.org (parse) Democratic Front for the Liberation of P...
en.wikipedia.org (parse) Demographics of Lebanon
en.wikipedia.org (imageinfo) File:Lebanon single age population p...
en.wikipedia.org (parse) Displaced persons
en.wikipedia.org (parse) El Al Flight 253 attack
en.wikipedia.org (imageinfo) File:OrlyAirport1965-Boeing707-EL-AL.jpg
en.wikipedia.org (parse) El Al Flight 426 hijacking
en.wikipedia.org (imageinfo) File:El Al 707 at Zurich 1982.jpg
en.wikipedia.org (parse) El Al Flight 432 attack
en.

-----------------
name:  Haifa
subdivision_type:  ['Country']
subdivision_name:  ['Flagu', 'Israel']
name ar:  حيفا 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
Cannot find the link: http://en.wikipedia.org/wiki/Flagu
subdivision_name:  Flagu
subdivision_name ar:  علم (مترجمه)
subdivision_name ar link:  -1
subdivision_name:  Israel
subdivision_name ar:  إسرائيل 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%A5%D8%B3%D8%B1%D8%A7%D8%A6%D9%8A%D9%84
END


en.wikipedia.org (parse) Hezbollah
en.wikipedia.org (parse) Hussein of Jordan
en.wikipedia.org (imageinfo) File:Hussein of Jordan in 1997.jpg
en.wikipedia.org (parse) ISBN (identifier)
en.wikipedia.org (imageinfo) File:EAN-13-ISBN-13.svg
en.wikipedia.org (parse) Iran


...

-----------------
name:  Tyre
subdivision_type:  ['Country']
subdivision_name:  ['Flag', 'Lebanon']
name ar:  صور (توضيح) 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Flag
subdivision_name ar:  علم (راية) 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_(%D8%B1%D8%A7%D9%8A%D8%A9)
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) Tyre (Lebanon)


-----------------
name:  Tyre
subdivision_type:  ['Country']
subdivision_name:  ['Flag', 'Lebanon']
name ar:  صور (توضيح) 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Flag
subdivision_name ar:  علم (راية) 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_(%D8%B1%D8%A7%D9%8A%D8%A9)
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) Yasser Arafat
en.wikipedia.org (imageinfo) File:Flickr - Government Press Offic...
en.wikipedia.org (parse) Yassir Arafat
en.wikipedia.org (imageinfo) File:Flickr - Government Press Offic...
en.wikipedia.org (parse) Yitzhak Navon
en.wikipedia.org (imageinfo) File:Portrait of MP Yitzhak Navon.jpg


skipping Wikipedia:Protection policy
skipping Wikipedia:Verifiability
skipping Template:Campaignbox Palestinian insurgency in South Lebanon
skipping Template:Country data Israel
skipping Template:Country data PLO
skipping Template:Country data Syria
skipping Template:Country data USSR
skipping Template talk:Campaignbox Palestinian insurgency in South Lebanon
skipping Category:Use dmy dates from November 2020
skipping Category:Wikipedia articles needing factual verification from November 2011


In [19]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,[Beirut],Country,Lebanon
1,Haifa,Country,"Flagu,Israel"
2,[Iran],-,-
3,[Israel],-,-
4,[Jordan],-,-
5,[Lebanon],-,-
6,Nahariyya,Country,ISR
7,[the Soviet Union],-,-
8,[Syria],-,-
9,Tunis,Country,Tunisia


In [20]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,لبنان,بلد,بيروت
1,"علم (مترجمه),إسرائيل",بلد,حيفا
2,-,-,إيران
3,-,-,إسرائيل
4,-,-,الأردن
5,-,-,لبنان
6,,بلد,نهاريا
7,-,-,الاتحاد السوفيتي
8,-,-,سوريا
9,تونس,بلد,تونس (مدينة)


In [21]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')